In [1]:
import pandas as pd
import geopandas as gpd
from tobler.area_weighted import area_interpolate
import numpy as np

/home/maciek/.pyenv/versions/3.8.12/envs/school-map-project/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
umwelt = gpd.read_file('../raw_data/Maps/umwelt.shp').drop(columns=['plr_name', 'status', 'anz_bel', 'wohnlage'])
building_age = gpd.read_file('../raw_data/Maps/building_age.shp')

In [3]:
umwelt.laerm = umwelt.laerm.replace(np.nan, umwelt.laerm.mode()[0])
umwelt.gruen = umwelt.gruen.replace(np.nan, umwelt.gruen.mode()[0])

In [4]:
pr2021 = gpd.read_file('../raw_data/Maps/pr_2021.shp')
pr2021['Nummer'] = pd.to_numeric(pr2021.PLR_ID)

In [118]:
umwelt_sjoined = gpd.sjoin(umwelt, pr2021, how='right').dissolve(by='PLR_ID').

In [121]:
umwelt_sjoined.reset_index(inplace=True)

In [13]:
building_age.x2011_2015 = building_age.x2011_2015.replace([np.nan, '1 - 3'], [0,2]).astype(float)

In [14]:
sjoined_right = gpd.sjoin(building_age, pr2021, how='right')

In [16]:
def get_agg_dict():
    d = {}
    for x in ['PLR_ID', 'BZR_ID', 'PGR_ID']:
        d[x] = 'first'
    for x in ['x_bis_1900', 'x1901_1910', 'x1911_1920', 'x1921_1930',
       'x1931_1940', 'x1941_1950', 'x1951_1960', 'x1961_1970', 'x1971_1980',
       'x1981_1990', 'x1991_2000', 'x2001_2010', 'x2011_2015', 'ew2015']:
        d[x] = 'sum'
    return d

In [17]:
agg_dict = get_agg_dict()

In [18]:
bsjr = sjoined_right.dissolve(by='PLR_ID', aggfunc=agg_dict).reset_index(drop=True)

In [155]:
merged = bsjr.merge(umwelt_sjoined, on='PLR_ID', how='left')
merged.drop(columns=['geometry_y', 'PLR_NAME', 'BZR_ID_y', 'BZR_NAME',
       'PGR_ID_y', 'PGR_NAME', 'BEZ', 'FINHALT', 'STAND', 'Nummer'], inplace=True)
merged['geometry'] = merged['geometry_x']
merged.drop(columns='geometry_x', inplace=True)

In [161]:
merged['laerm'] = merged['laerm'].map({'mittel':1, 'niedrig - sehr niedrig':0, 'hoch': 2, 'sehr hoch': 3})
merged['bio'] = merged['bio'].map({'mittel':1, 'gering':0, 'hoch': 2})
merged['luft'] = merged['luft'].map({'mittel':1, 'gering':0, 'hoch': 2})
merged['gruen'] = merged['gruen'].map({'schlecht, sehr schlecht': 0, 'mittel': 1, 'gut, sehr gut': 2})

In [170]:
gpd.GeoDataFrame(merged).to_file('../raw_data/Maps/umwelt_and_buildings.shp')